# [ 2개의 층 ]

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf

print("Keras:", keras.__version__)
print("TensorFlow:", tf.__version__)
print("Backend:", keras.config.backend())

2025-11-20 21:52:38.650319: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 21:52:38.659496: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-20 21:52:38.777777: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-20 21:52:38.854565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763643158.930968     919 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763643158.95

Keras: 3.8.0
TensorFlow: 2.18.0
Backend: tensorflow


In [2]:
# 케라스 API를 사용해서 패션 MNIST 데이터셋 불러온다.
import keras
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

In [3]:
# 이미지의 픽셀값을 0~255 범위에서 0~1사이로 변환하고, 28 * 28 크기의 2차원배열을 784 크기의 1차원 배열로 펼친다.
# 다음에 train_test_split()함수로 훈련 세트와 검증 세트로 나눈다.

from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

* 인공 신경망 모델에 층을 2개 추가하기
1. 1절을 만든 신경망 모델과 다른 점은 입력층과 출력층 사이에 밀집층이 추가된 것
2. 이렇게 입력층과 출력층 사이에 있는 모든 층을 '은닉층'이라고 부른다.
3. 활성화 함수: 신경망 층의 선형 방정식의 계산 값에 적용하는 함수
4. 은닉층의 활성화 함수는 비교적 자유로움(시그모이드, 렐루 함수 등을 사용)
5. 분류 문제를 위한 신경망의 출력층에는 시그모이드 함수나 소프트맥수 함수를 활성화 함수로 사용

In [4]:
# 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥수 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어 보겠습니다.

inputs = keras.layers.Input(shape=(784,))
dense1 = keras.layers.Dense(100, activation = 'sigmoid')
dense2 = keras.layers.Dense(10, activation = 'softmax')

1. dense1이 은닉층이고 100개의 뉴런을 가진 밀집층이다.
2. 한 가지 제약 사항이 있다면 적어도 출력층의 뉴련 보다는 많게 많들어야 한다.
3. dense2가 출력층이다.

* 심층 신경망 만들기
1. 앞에서 만든 inputs와 dense1, dense2 객체를 Sequential 클래스에 추가하여 심층 신경망을 만들어 본다.

In [5]:
model = keras.Sequential([inputs, dense1,dense2])

W0000 00:00:1763643163.771180     919 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


> Sequential 클래스의 객체를 만들 때는 여러 개의 층을 추가하려면 이와 같이 리스트에 계속 필요한 층을 추가하면 된다.
* 주의할 것은 입력층을 맨 앞에 두고 출력층을 가장 마지막에 두어야 한다는 것이다!!

In [6]:
# 케라스는 모델의 summary() 메서드를 호출하면 층에 대한 정보를 얻을 수 있다.

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

> 첫 번째 _ None
1. 출력 크기를 보면 (None, 100)이다.
2. 첫 번째 차원은 샘플의 개수를 나타낸다. ---> 샘플의 개수가 아직 정의되어 있지 않기에 None이다.
3. 케라스 fit() 메서드에서는 미니배치 경사 하강법을 실시한다.
4. fit() 메서드에서 batch_size 매개변수로 바꿀 수 있다. ---> 샘플 개수를 고정하지 않고 어떤 배치 크기에도 대응할 수 있도록 None으로 설정한다.

> 두 번째 _ 100
1. 은닉층의 뉴런 개수를 100개로 두었으니 100개의 출력이 나온다.
2. 샘플 마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축이 된 것이다.

> 마지막 _ 모델 파라미터 개수
1. 이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있다.
2. 뉴런 마다 1개의 절편이 있다.

> 층을 추가하는 다른 방법
1. Sequential 클래스의 생성자 안에서 바로 클래스의 객체를 만드는 경우

In [7]:
# Dense 층의 name 매개변수에 층의 이름을 '은닉층'과 '출력층'을고 각각 지정했다.

model = keras.Sequential([
    keras.layers.Input(shape=(784,)),
    keras.layers.Dense(100, activation = 'sigmoid', name = '은닉층'),
    keras.layers.Dense(10, activation = 'softmax', name = '출력층')
], name = '패션 MNIST 모델')

In [8]:
model.summary()

Model: "패션 MNIST 모델"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 은닉층 (Dense)                  │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ 출력층 (Dense)                  │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

> 위 방법은 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어진다. 또 조건에 따라 층을 추가할 수도 없다.

> 그래서 Sequential 클래스에서 층을 추가할 때 가장 널리 사용하는 방법의 모델은 add() 메서드입니다.

In [9]:
# add() 메서드 호출.

model = keras.Sequential()
model.add(keras.layers.Input(shape=(784,)))
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [10]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# 모델 훈련

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7497 - loss: 0.7784
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8511 - loss: 0.4216
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8614 - loss: 0.3788
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8706 - loss: 0.3604
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8762 - loss: 0.3386


# [ 렐루 함수 ]
> 시그모이드 함수의 단점 --> 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있어 올바른 출력을 만드는데 신속하게 대응하지 못함
* 이를 개선하기 위한게 렐루 함수이다.
* 렐루 함수는 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수이 경우에는 0으로 만든다.
* 렐루 함수는 심층 신경망에서 뛰어나다.

> 렐루 함수 max(0, z)와 같이 사용할 경우
* z > 0 : z를 출력
* z < 0 : 0을 출력

In [12]:
# Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다. ==> 입력에 곱해지는 가중치나 절편이 없다.
# 하지만 Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라 부른다.

model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [13]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

* 위에 첫 번째 Input() 함수의 shape 매개변수에 입력의 크기를 (784,)가 아니라 이미지 크기인 (28,28)로 지정했다.
* 다음에 등장하는 Flatten 층이 1차원으로 펼쳐 줄 것이므로 입력층에는 2차원 입력을 받겠다는 의미이다.
* 첫 번째 Dense 층의 활성화 함수를 'relu'로 바꾼 것을 보자. ==> 이 신경망을 깊이가 3인 신경망이라고 부르지 않는다. ==> Flatten 클래스는 학습하는 층이 아니기 때문이다.

> 입력 데이터에 대한 전처리 과정을 가능한 모델에 포함시키는 것이 케라스API의 철학 중 하나이다.

In [14]:
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [15]:
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7667 - loss: 0.6680
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8587 - loss: 0.3992
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8736 - loss: 0.3523
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8808 - loss: 0.3326
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8872 - loss: 0.3133


In [16]:
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8722 - loss: 0.3625


[0.36429011821746826, 0.8730833530426025]

# [ 옵티마이저 ]

* 추가할 은닉층의 개수는 모델이 학습하는 것이 아니라 우리가 지정해 주어야 할 하이퍼파라미터이다.
* 은닉층의 뉴런 개수도 하이퍼파라미터이다. + 활성화 함수

> 가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD이다.

In [17]:
# compile() 메서드의 optimizer 매개변수를 'sgd'로 지정한다.

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
# SGD 클래스 학습률의 기본값은 0.01이다. 이를 바꾸고 싶다면 다음과 같이 원하는 학습률을 learning_rate 매개변수에 지정하여 사용한다.

sgd = keras.optimizers.SGD(learning_rate=0.1)

* 기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공한다.
* SGD 클래스의 momentum 매개변수의 기본값은 0이다.
* 이를 0보다 큰 값으로 지정하면 마치 이전의 그레이디언트 가속도 처럼 사용하는 '모멘텀 최적화'를 사용한다. (보통 momentum 매개변수: 0.9)

In [20]:
# SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 네스테로프 모멘텀 최적화를 사용한다.

sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

> 적응적 학습률: 모델이 최적점에 가까이 갈수록 학습률을 낮춘다. ==> 안정적으로 최적점에 수렴할 가능성이 높아진다.

In [21]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

* RMSprop의 장점을 접목한 것이 Adam이다.
* Adam 클래스도 keras.optimizers 패키지 아래에 있다.
* 적응적 학습률을 사용하는 이 3개의 클래스는 learning_rate 매개변수의 기본값으로 모두 0.001을 사용한다.

In [23]:
# Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련

model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [24]:
# compile() 메서드의 optimizer를 'adam'으로 설정하고 5번의 에포크 동안 훈련한다.

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7763 - loss: 0.6471
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8585 - loss: 0.3986
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8715 - loss: 0.3547
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8808 - loss: 0.3229
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8865 - loss: 0.3040


In [25]:
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8799 - loss: 0.3357


[0.3330775201320648, 0.8815000057220459]